## Hitchhiker's Guide to Hyperspace - An Indexing Subsystem for Apache Spark™
Hyperspace introduces the ability for Apache Spark™ users to create indexes on their datasets (e.g., CSV, JSON, Parquet etc.) and leverage them for potential query and workload acceleration.

In this notebook, we highlight the basics of Hyperspace, emphasizing on its simplicity and shows how it can be used by just anyone.

**Disclaimer**: Hyperspace helps accelerate your workloads/queries under two circumstances:

  1. Queries contain filters on predicates with high selectivity (e.g., you want to select 100 matching rows from a million candidate rows)
  2. Queries contain a join that requires heavy-shuffles (e.g., you want to join a 100 GB dataset with a 10 GB dataset)

You may want to carefully monitor your workloads and determine whether indexing is helping you on a case-by-case basis.

## Setup
To begin with, let's start a new Spark™ session. Since this notebook is a tutorial merely to illustrate what Hyperspace can offer, we will make a configuration change that allow us to highlight what Hyperspace is doing on small datasets. By default, Spark™ uses *broadcast join* to optimize join queries when the data size for one side of join is small (which is the case for the sample data we use in this tutorial). Therefore, we disable broadcast joins so that later when we run join queries, Spark™ uses *sort-merge* join. This is mainly to show how Hyperspace indexes would be used at scale for accelerating join queries.

The output of running the cell below shows a reference to the successfully created Spark™ session and prints out '-1' as the value for the modified join config which indicates that broadcast join is successfully disabled.

In [46]:
// Disable BroadcastHashJoin, so Spark™ will use standard SortMergeJoin. Currently hyperspace indexes utilize SortMergeJoin to speed up query.
spark.Conf().Set("spark.sql.autoBroadcastJoinThreshold", -1);

// Verify that BroadcastHashJoin is set correctly 
Console.WriteLine(spark.Conf().Get("spark.sql.autoBroadcastJoinThreshold"));

-1

## Data Preparation

To prepare our environment, we will create sample data records and save them as parquet data files. While we use Parquet for illustration, you can use other formats such as CSV. In the subsequent cells, we will also demonstrate how you can create several Hyperspace indexes on this sample dataset and how one can make Spark™ use them when running queries. 

Our example records correspond to two datasets: *department* and *employee*. You should configure "empLocation" and "deptLocation" paths so that on the storage account they point to your desired location to save generated data files. 

The output of running below cell shows contents of our datasets as lists of triplets followed by references to dataFrames created to save the content of each dataset in our preferred location.

In [47]:
using Microsoft.Spark.Sql.Types;

// Sample department records
var departments = new List<GenericRow>()
{
    new GenericRow(new object[] {10, "Accounting", "New York"}),
    new GenericRow(new object[] {20, "Research", "Dallas"}),
    new GenericRow(new object[] {30, "Sales", "Chicago"}),
    new GenericRow(new object[] {40, "Operations", "Boston"})
};

// Sample employee records
var employees = new List<GenericRow>() {
      new GenericRow(new object[] {7369, "SMITH", 20}),
      new GenericRow(new object[] {7499, "ALLEN", 30}),
      new GenericRow(new object[] {7521, "WARD", 30}),
      new GenericRow(new object[] {7566, "JONES", 20}),
      new GenericRow(new object[] {7698, "BLAKE", 30}),
      new GenericRow(new object[] {7782, "CLARK", 10}),
      new GenericRow(new object[] {7788, "SCOTT", 20}),
      new GenericRow(new object[] {7839, "KING", 10}),
      new GenericRow(new object[] {7844, "TURNER", 30}),
      new GenericRow(new object[] {7876, "ADAMS", 20}),
      new GenericRow(new object[] {7900, "JAMES", 30}),
      new GenericRow(new object[] {7934, "MILLER", 10}),
      new GenericRow(new object[] {7902, "FORD", 20}),
      new GenericRow(new object[] {7654, "MARTIN", 30})
};

// Save sample data in the Parquet format
var departmentSchema = new StructType(new List<StructField>()
{
    new StructField("deptId", new IntegerType()),
    new StructField("deptName", new StringType()),
    new StructField("location", new StringType())
});
var employeeSchema = new StructType(new List<StructField>()
{
    new StructField("empId", new IntegerType()),
    new StructField("empName", new StringType()),
    new StructField("deptId", new IntegerType())
});

DataFrame empData = spark.CreateDataFrame(employees, employeeSchema); 
DataFrame deptData = spark.CreateDataFrame(departments, departmentSchema); 

string empLocation = "/<yourpath>/employees.parquet";       //TODO ** customize this location path **
string deptLocation = "/<yourpath>/departments.parquet";     //TODO ** customize this location path **
empData.Write().Mode("overwrite").Parquet(empLocation);
deptData.Write().Mode("overwrite").Parquet(deptLocation);

Let's verify the contents of parquet files we created above to make sure they contain expected records in correct format. We later use these data files to create Hyperspace indexes and run sample queries.

Running below cell, the output displays the rows in employee and department dataFrames in a tabular form. There should be 14 employees and 4 departments, each matching with one of triplets we created in the previous cell.

In [48]:
// empLocation and deptLocation are the user defined locations above to save parquet files
DataFrame empDF = spark.Read().Parquet(empLocation);
DataFrame deptDF = spark.Read().Parquet(deptLocation);

// Verify the data is available and correct
empDF.Show();
deptDF.Show();

+-----+-------+------+
|empId|empName|deptId|
+-----+-------+------+
| 7499|  ALLEN|    30|
| 7521|   WARD|    30|
| 7369|  SMITH|    20|
| 7844| TURNER|    30|
| 7876|  ADAMS|    20|
| 7900|  JAMES|    30|
| 7934| MILLER|    10|
| 7566|  JONES|    20|
| 7698|  BLAKE|    30|
| 7782|  CLARK|    10|
| 7788|  SCOTT|    20|
| 7839|   KING|    10|
| 7902|   FORD|    20|
| 7654| MARTIN|    30|
+-----+-------+------+

+------+----------+--------+
|deptId|  deptName|location|
+------+----------+--------+
|    10|Accounting|New York|
|    40|Operations|  Boston|
|    20|  Research|  Dallas|
|    30|     Sales| Chicago|
+------+----------+--------+

## Hello Hyperspace Index!
Hyperspace lets users create indexes on records scanned from persisted data files. Once successfully created, an entry corresponding to the index is added to the Hyperspace's metadata. This metadata is later used by Apache Spark™'s optimizer (with our extensions) during query processing to find and use proper indexes. 

Once indexes are created, users can perform several actions:
  - **Refresh** If the underlying data changes, users can refresh an existing index to capture that. 
  - **Delete** If the index is not needed, users can perform a soft-delete i.e., index is not physically deleted but is marked as 'deleted' so it is no longer used in your workloads.
  - **Vacuum** If an index is no longer required, users can vacuum it which forces a physical deletion of the index contents and associated metadata completely from Hyperspace's metadata.

Below sections show how such index management operations can be done in Hyperspace.

First, we need to import the required libraries and create an instance of Hyperspace. We later use this instance to invoke different Hyperspace APIs to create indexes on our sample data and modify those indexes.

Output of running below cell shows a reference to the created instance of Hyperspace.

In [49]:
// Create an instance of Hyperspace
using Microsoft.Spark.Extensions.Hyperspace;

Hyperspace hyperspace = new Hyperspace(spark);

### Create Indexes
To create a Hyperspace index, the user needs to provide 2 pieces of information:
* An Apache Spark™ DataFrame which references the data to be indexed.
* An index configuration object: IndexConfig, which specifies the *index name*, *indexed* and *included* columns of the index. 

We start by creating three Hyperspace indexes on our sample data: two indexes on the department dataset named "deptIndex1" and "deptIndex2", and one index on the employee dataset named 'empIndex'. 
For each index, we need a corresponding IndexConfig to capture the name along with columns lists for the indexed and included columns. Running below cell creates these indexConfigs and its output lists them.

**Note**: An *index column* is a column that appears in your filters or join conditions. An *included column* is a column that appears in your select/project.

For instance, in the following query:
```sql
SELECT X
FROM Table
WHERE Y = 2
```
X can be an *index column* and Y can be an *included column*.

In [50]:
// Create index configurations
using Microsoft.Spark.Extensions.Hyperspace.Index;

var empIndexConfig = new IndexConfig("empIndex", new string[] {"deptId"}, new string[] {"empName"});
var deptIndexConfig1 = new IndexConfig("deptIndex1", new string[] {"deptId"}, new string[] {"deptName"});
var deptIndexConfig2 = new IndexConfig("deptIndex2", new string[] {"location"}, new string[] {"deptName"});

Now, we create three indexes using our index configurations. For this purpose, we invoke "createIndex" command on our Hyperspace instance. This command requires an index configuration and the dataFrame containing rows to be indexed.
Running below cell creates three indexes.


In [51]:
// Create indexes from configurations
hyperspace.CreateIndex(empDF, empIndexConfig);
hyperspace.CreateIndex(deptDF, deptIndexConfig1);
hyperspace.CreateIndex(deptDF, deptIndexConfig2);

### List Indexes

Below code shows how a user can list all available indexes in a Hyperspace instance. It uses "indexes" API which returns information about existing indexes as a Spark™'s DataFrame so you can perform additional operations. For instance, you can invoke valid operations on this DataFrame for checking its content or analyzing it further (for example filtering specific indexes or grouping them according to some desired property). 

Below cell uses DataFrame's 'show' action to fully print the rows and show details of our indexes in a tabular form. For each index, we can see all information Hyperspace has stored about it in the metadata. You will immediately notice the following:
  - "config.indexName", "config.indexedColumns", "config.includedColumns" and "status.status" are the fields that a user normally refers to. 
  - "dfSignature" is automatically generated by Hyperspace and is unique for each index. Hyperspace uses this signature internally to maintain the index and exploit it at query time. 
  
In the output below, all three indexes should have "ACTIVE" as status and their name, indexed columns, and included columns should match with what we defined in index configurations above.


In [52]:
hyperspace.Indexes().Show();

+----------+--------------+---------------+----------+--------------------+--------------------+------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation| state|
+----------+--------------+---------------+----------+--------------------+--------------------+------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://data@arca...|ACTIVE|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://data@arca...|ACTIVE|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://data@arca...|ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+------+

### Delete Indexes
A user can drop an existing index by using the "deleteIndex" API and providing the index name. Index deletion does a soft delete: It mainly updates index's status in the Hyperspace metadata from "ACTIVE" to "DELETED". This will exclude the dropped index from any future query optimization and Hyperspace no longer picks that index for any query. However, index files for a deleted index still remain available (since it is a soft-delete), so that the index could be restored if user asks for.

Below cell deletes index with name "deptIndex2" and lists Hyperspace metadata after that. The output should be similar to above cell for "List Indexes" except for "deptIndex2" which now should have its status changed into "DELETED".

In [53]:
hyperspace.DeleteIndex("deptIndex2");

hyperspace.Indexes().Show();

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://data@arca...| ACTIVE|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://data@arca...|DELETED|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://data@arca...| ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+

### Restore Indexes
A user can use the "restoreIndex" API to restore a deleted index. This will bring back the latest version of index into ACTIVE status and makes it usable again for queries. Below cell shows an example of "restoreIndex" usage. We delete "deptIndex1" and restore it. The output shows "deptIndex1" first went into the "DELETED" status after invoking "deleteIndex" command and came back to the "ACTIVE" status after calling "restoreIndex".


In [54]:
hyperspace.DeleteIndex("deptIndex1");

hyperspace.Indexes().Show();

hyperspace.RestoreIndex("deptIndex1");

hyperspace.Indexes().Show();

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://data@arca...|DELETED|
|deptIndex2|    [location]|     [deptName]|       200|{"type":"struct",...|abfss://data@arca...|DELETED|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://data@arca...| ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+-------+

+----------+--------------+---------------+----------+--------------------+--------------------+-------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation|  state|
+----------+--------------+---------------+----------+

### Vacuum Indexes
The user can perform a hard-delete i.e., fully remove files and the metadata entry for a deleted index using "vacuumIndex" command. Once done, this action is irreversible as it physically deletes all the index files (which is why it is a hard-delete).
 
The cell below vacuums the "deptIndex2" index and shows Hyperspace metadata after vaccuming. You should see metadata entries for two indexes "deptIndex1" and "empIndex" both with "ACTIVE" status and no entry for "deptIndex2".

In [55]:
hyperspace.VacuumIndex("deptIndex2");

hyperspace.Indexes().Show();

+----------+--------------+---------------+----------+--------------------+--------------------+------+
|      name|indexedColumns|includedColumns|numBuckets|              schema|       indexLocation| state|
+----------+--------------+---------------+----------+--------------------+--------------------+------+
|deptIndex1|      [deptId]|     [deptName]|       200|{"type":"struct",...|abfss://data@arca...|ACTIVE|
|  empIndex|      [deptId]|      [empName]|       200|{"type":"struct",...|abfss://data@arca...|ACTIVE|
+----------+--------------+---------------+----------+--------------------+--------------------+------+

## Enable/Disable Hyperspace

Hyperspace provides APIs to enable or disable index usage with Spark™.

  - By using "enableHyperspace" command, Hyperspace optimization rules become visible to the Apache Spark™ optimizer and they will exploit existing Hyperspace indexes to optimize user queries.
  - By using "disableHyperspace' command, Hyperspace rules no longer apply during query optimization. You should note that disabling Hyperspace has no impact on created indexes as they remain intact.

Below cell shows how you can use these commands to enable or disable hyperspace. The output simply shows a reference to the existing Spark™ session whose configuration is updated.

In [56]:
// Enable Hyperspace
spark.EnableHyperspace();

// Disable Hyperspace
spark.DisableHyperspace();

## Index Usage
In order to make Spark™ use Hyperspace indexes during query processing, the user needs to make sure that Hyperspace is enabled. 

The cell below enables Hyperspace and creates two DataFrames containing our sample data records which we use for running example queries. For each DataFrame, a few sample rows are printed.

In [57]:
// Enable Hyperspace
spark.EnableHyperspace();

DataFrame empDFrame = spark.Read().Parquet(empLocation);
DataFrame deptDFrame = spark.Read().Parquet(deptLocation);

empDFrame.Show(5);
deptDFrame.Show(5);

+-----+-------+------+
|empId|empName|deptId|
+-----+-------+------+
| 7499|  ALLEN|    30|
| 7521|   WARD|    30|
| 7369|  SMITH|    20|
| 7844| TURNER|    30|
| 7876|  ADAMS|    20|
+-----+-------+------+
only showing top 5 rows

+------+----------+--------+
|deptId|  deptName|location|
+------+----------+--------+
|    10|Accounting|New York|
|    40|Operations|  Boston|
|    20|  Research|  Dallas|
|    30|     Sales| Chicago|
+------+----------+--------+

# Hyperspace's Index Types

Currently, Hyperspace has rules to exploit indexes for two groups of queries: 
* Selection queries with lookup or range selection filtering predicates.
* Join queries with an equality join predicate (i.e. Equi-joins).


## Indexes for Accelerating Filters

Our first example query does a lookup on department records (see below cell). In SQL, this query looks as follows:

```sql
SELECT deptName 
FROM departments
WHERE deptId = 20
```

The output of running the cell below shows: 
- query result, which is a single department name.
- query plan that Spark™ used to run the query. 

In the query plan, the "FileScan" operator at the bottom of the plan shows the datasource where the records were read from. The location of this file indicates the path to the latest version of the "deptIndex1" index. This shows  that according to the query and using Hyperspace optimization rules, Spark™ decided to exploit the proper index at runtime.


In [58]:
// Filter with equality predicate
DataFrame eqFilter = deptDFrame.Filter("deptId = 20").Select("deptName");
eqFilter.Show();

eqFilter.Explain(true);

+--------+
|deptName|
+--------+
|Research|
+--------+

== Parsed Logical Plan ==
'Project [unresolvedalias('deptName, None)]
+- Filter (deptId#1075 = 20)
   +- Relation[deptId#1075,deptName#1076,location#1077] parquet

== Analyzed Logical Plan ==
deptName: string
Project [deptName#1076]
+- Filter (deptId#1075 = 20)
   +- Relation[deptId#1075,deptName#1076,location#1077] parquet

== Optimized Logical Plan ==
Project [deptName#1076]
+- Filter (isnotnull(deptId#1075) && (deptId#1075 = 20))
   +- Relation[deptId#1075,deptName#1076,location#1077] parquet

== Physical Plan ==
*(1) Project [deptName#1076]
+- *(1) Filter (isnotnull(deptId#1075) && (deptId#1075 = 20))
   +- *(1) FileScan parquet [deptId#1075,deptName#1076] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://data@mydatalake.dfs.core.windows.net/<yourpath>/departments.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(deptId), EqualTo(deptId,20)], ReadSchema: struct<deptId:int,deptName:string>

Our second example is a range selection query on department records. In SQL, this query looks as follows:

```sql
SELECT deptName 
FROM departments
WHERE deptId > 20
```
Similar to our first example, the output of the cell below shows the query results (names of two departments) and the query plan. The location of data file in the FileScan operator shows that 'deptIndex1" was used to run the query.   


In [59]:
// Filter with range selection predicate
DataFrame rangeFilter = deptDFrame.Filter("deptId > 20").Select("deptName");
rangeFilter.Show();

rangeFilter.Explain(true);

+----------+
|  deptName|
+----------+
|Operations|
|     Sales|
+----------+

== Parsed Logical Plan ==
'Project [unresolvedalias('deptName, None)]
+- Filter (deptId#1075 > 20)
   +- Relation[deptId#1075,deptName#1076,location#1077] parquet

== Analyzed Logical Plan ==
deptName: string
Project [deptName#1076]
+- Filter (deptId#1075 > 20)
   +- Relation[deptId#1075,deptName#1076,location#1077] parquet

== Optimized Logical Plan ==
Project [deptName#1076]
+- Filter (isnotnull(deptId#1075) && (deptId#1075 > 20))
   +- Relation[deptId#1075,deptName#1076] parquet

== Physical Plan ==
*(1) Project [deptName#1076]
+- *(1) Filter (isnotnull(deptId#1075) && (deptId#1075 > 20))
   +- *(1) FileScan parquet [deptId#1075,deptName#1076] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://data@mydatalake.dfs.core.windows.net/synapse/workspaces/myworkspace/war..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId), GreaterThan(deptId,20)], ReadSchema: struct<deptId:int,deptName:s

Our third example is a query joining department and employee records on the department id. The equivalent SQL statement is shown below:

```sql
SELECT employees.deptId, empName, departments.deptId, deptName
FROM   employees, departments 
WHERE  employees.deptId = departments.deptId
```

The output of running the cell below shows the query results which are the names of 14 employees and the name of department each employee works in. The query plan is also included in the output. Notice how the file locations for two FileScan operators shows that Spark™ used "empIndex" and "deptIndex1" indexes to run the query.   


In [60]:
// Join
DataFrame eqJoin =
      empDFrame
      .Join(deptDFrame, empDFrame.Col("deptId") == deptDFrame.Col("deptId"))
      .Select(empDFrame.Col("empName"), deptDFrame.Col("deptName"));

eqJoin.Show();

eqJoin.Explain(true);

+-------+----------+
|empName|  deptName|
+-------+----------+
|  ADAMS|  Research|
|  JONES|  Research|
|  SMITH|  Research|
|  SCOTT|  Research|
|   FORD|  Research|
| MILLER|Accounting|
|  CLARK|Accounting|
|   KING|Accounting|
|  ALLEN|     Sales|
|   WARD|     Sales|
| TURNER|     Sales|
|  BLAKE|     Sales|
|  JAMES|     Sales|
| MARTIN|     Sales|
+-------+----------+

== Parsed Logical Plan ==
Project [empName#1070, deptName#1076]
+- Join Inner, (deptId#1071 = deptId#1075)
   :- Relation[empId#1069,empName#1070,deptId#1071] parquet
   +- Relation[deptId#1075,deptName#1076,location#1077] parquet

== Analyzed Logical Plan ==
empName: string, deptName: string
Project [empName#1070, deptName#1076]
+- Join Inner, (deptId#1071 = deptId#1075)
   :- Relation[empId#1069,empName#1070,deptId#1071] parquet
   +- Relation[deptId#1075,deptName#1076,location#1077] parquet

== Optimized Logical Plan ==
Project [empName#1070, deptName#1076]
+- Join Inner, (deptId#1071 = deptId#1075)
   :- Proje

## Support for SQL Semantics

The index usage is transparent to whether the user uses DataFrame API or Spark™ SQL. The following example shows the same join example as before, in sql form, showing the use of indexes if applicable.

In [61]:
empDFrame.CreateOrReplaceTempView("EMP");
deptDFrame.CreateOrReplaceTempView("DEPT");

var joinQuery = spark.Sql("SELECT EMP.empName, DEPT.deptName FROM EMP, DEPT WHERE EMP.deptId = DEPT.deptId");

joinQuery.Show();
joinQuery.Explain(true);

+-------+----------+
|empName|  deptName|
+-------+----------+
|  ADAMS|  Research|
|  JONES|  Research|
|  SMITH|  Research|
|  SCOTT|  Research|
|   FORD|  Research|
| MILLER|Accounting|
|  CLARK|Accounting|
|   KING|Accounting|
|  ALLEN|     Sales|
|   WARD|     Sales|
| TURNER|     Sales|
|  BLAKE|     Sales|
|  JAMES|     Sales|
| MARTIN|     Sales|
+-------+----------+

== Parsed Logical Plan ==
'Project ['EMP.empName, 'DEPT.deptName]
+- 'Filter ('EMP.deptId = 'DEPT.deptId)
   +- 'Join Inner
      :- 'UnresolvedRelation `EMP`
      +- 'UnresolvedRelation `DEPT`

== Analyzed Logical Plan ==
empName: string, deptName: string
Project [empName#1070, deptName#1076]
+- Filter (deptId#1071 = deptId#1075)
   +- Join Inner
      :- SubqueryAlias `emp`
      :  +- Relation[empId#1069,empName#1070,deptId#1071] parquet
      +- SubqueryAlias `dept`
         +- Relation[deptId#1075,deptName#1076,location#1077] parquet

== Optimized Logical Plan ==
Project [empName#1070, deptName#1076]
+- Join

## Explain API
Indexes are great but how do you know if they are being used? Hyperspace allows users to compare their original plan vs the updated index-dependent plan before running their query. You have an option to choose from html/plaintext/console mode to display the command output. 

The following cell shows an example with HTML. The highlighted section represents the difference between original and updated plans along with the indexes being used.

In [62]:
spark.Conf().Set("spark.hyperspace.explain.displayMode", "html");
spark.Conf().Set("spark.hyperspace.explain.displayMode.highlight.beginTag", "<b style=\"background:LightGreen\">");
spark.Conf().Set("spark.hyperspace.explain.displayMode.highlight.endTag", "</b>");

hyperspace.Explain(eqJoin, false, input => DisplayHTML(input));

============================================================= Plan with indexes: ============================================================= Project [empName#1070, deptName#1076] +- SortMergeJoin [deptId#1071], [deptId#1075], Inner :- *(1) Project [empName#1070, deptId#1071] : +- *(1) Filter isnotnull(deptId#1071) : +- *(1) FileScan parquet [deptId#1071,empName#1070] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://data@mydatalake.dfs.core.windows.net/synapse/workspaces/myworkspace/war..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 +- *(2) Project [deptId#1075, deptName#1076] +- *(2) Filter isnotnull(deptId#1075) +- *(2) FileScan parquet [deptId#1075,deptName#1076] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://data@mydatalake.dfs.core.windows.net/synapse/workspaces/myworkspace/war..., PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct , SelectedBucketsCount: 200 out of 200 ============================================================= Plan without indexes: ============================================================= Project [empName#1070, deptName#1076] +- SortMergeJoin [deptId#1071], [deptId#1075], Inner :- *(2) Sort [deptId#1071 ASC NULLS FIRST], false, 0 : +- Exchange hashpartitioning(deptId#1071, 200), [id=#956] : +- *(1) Project [empName#1070, deptId#1071] : +- *(1) Filter isnotnull(deptId#1071) : +- *(1) FileScan parquet [empName#1070,deptId#1071] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://data@mydatalake.dfs.core.windows.net/ /employees.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct +- *(4) Sort [deptId#1075 ASC NULLS FIRST], false, 0 +- Exchange hashpartitioning(deptId#1075, 200), [id=#962] +- *(3) Project [deptId#1075, deptName#1076] +- *(3) Filter isnotnull(deptId#1075) +- *(3) FileScan parquet [deptId#1075,deptName#1076] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://data@mydatalake.dfs.core.windows.net/ /departments.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(deptId)], ReadSchema: struct ============================================================= Indexes used: ============================================================= deptIndex1:abfss://data@mydatalake.dfs.core.windows.net/synapse/workspaces/myworkspace/warehouse/indexes/deptIndex1/v__=0 empIndex:abfss://data@mydatalake.dfs.core.windows.net/synapse/workspaces/myworkspace/warehouse/indexes/empIndex/v__=0

## Refresh Indexes
If the original data on which an index was created changes, then the index will no longer capture the latest state of data. The user can refresh such a stale index using "refreshIndex" command. This causes the index to be fully rebuilt and updates it according to the latest data records (don't worry, we will show you how to *incrementally refresh* your index in other notebooks).

The two cells below show an example for this scenario:
- First cell adds two more departments to the original departments data. It reads and prints list of departments to verify new departments are added correctly. The output shows 6 departments in total: four old ones and two new. Invoking "refreshIndex" updates "deptIndex1" so index captures new departments.
- Second cell runs our range selection query example. The results should now contain four departments: two are the ones, seen before when we ran the query above, and two are the new departments we just added.

In [63]:
var extraDepartments = new List<GenericRow>()
{
    new GenericRow(new object[] {50, "Inovation", "Seattle"}),
    new GenericRow(new object[] {60, "Human Resources", "San Francisco"})
};
	  
DataFrame extraDeptData = spark.CreateDataFrame(extraDepartments, departmentSchema);
extraDeptData.Write().Mode("Append").Parquet(deptLocation);

DataFrame deptDFrameUpdated = spark.Read().Parquet(deptLocation);

deptDFrameUpdated.Show(10);

hyperspace.RefreshIndex("deptIndex1");

+------+---------------+-------------+
|deptId|       deptName|     location|
+------+---------------+-------------+
|    60|Human Resources|San Francisco|
|    10|     Accounting|     New York|
|    50|      Inovation|      Seattle|
|    40|     Operations|       Boston|
|    20|       Research|       Dallas|
|    30|          Sales|      Chicago|
+------+---------------+-------------+

In [64]:
DataFrame newRangeFilter = deptDFrameUpdated.Filter("deptId > 20").Select("deptName");
newRangeFilter.Show();

newRangeFilter.Explain(true);

+---------------+
|       deptName|
+---------------+
|Human Resources|
|      Inovation|
|     Operations|
|          Sales|
+---------------+

== Parsed Logical Plan ==
'Project [unresolvedalias('deptName, None)]
+- Filter (deptId#1179 > 20)
   +- Relation[deptId#1179,deptName#1180,location#1181] parquet

== Analyzed Logical Plan ==
deptName: string
Project [deptName#1180]
+- Filter (deptId#1179 > 20)
   +- Relation[deptId#1179,deptName#1180,location#1181] parquet

== Optimized Logical Plan ==
Project [deptName#1180]
+- Filter (isnotnull(deptId#1179) && (deptId#1179 > 20))
   +- Relation[deptId#1179,deptName#1180,location#1181] parquet

== Physical Plan ==
*(1) Project [deptName#1180]
+- *(1) Filter (isnotnull(deptId#1179) && (deptId#1179 > 20))
   +- *(1) FileScan parquet [deptId#1179,deptName#1180] Batched: true, Format: Parquet, Location: InMemoryFileIndex[abfss://data@mydatalake.dfs.core.windows.net/<yourpath>/departments.parquet], PartitionFilters: [], PushedFilters: [IsNotNull(

In [65]:
// Clean-up the remaining indexes
hyperspace.DeleteIndex("empIndex");
hyperspace.DeleteIndex("deptIndex1");

hyperspace.VacuumIndex("empIndex");
hyperspace.VacuumIndex("deptIndex1");